In [9]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import plotly.express as px

In [10]:
from scripts.preproc import load_data, split_df, get_xy
from scripts.train import train_rfr, predict_val
from scripts.submit import get_submitable
train_cols = ['precipitation_amt_mm']
target_col = 'total_cases'
test_size = 0.2

# 1.Preprocessing

## 1.1 Train features and labes are merged, both train and test DFs are devided by cities

In [11]:
train_sj, train_iq, test_sj, test_iq = load_data()

## 1.2. Visualisation

In [12]:
fig = px.line(train_sj, x='date', y=["precipitation_amt_mm", 'station_avg_temp_c', 'total_cases'], title='San Juan')
fig.show()

fig = px.line(train_iq, x='date', y=["precipitation_amt_mm", 'station_avg_temp_c', 'total_cases'], title='Iquitos')
fig.show()

## 1.3. Spliting into training and validation sets

I suggest that at the end of preprocessing for either city we get **four** dataframes `(X + Y) × (train + test)`

In [13]:
X_train_sj, X_val_sj, y_train_sj, y_val_sj = get_xy(train_sj, test_size, train_cols, target_col)

X_train_iq, X_val_iq, y_train_iq, y_val_iq = get_xy(train_iq, test_size, train_cols, target_col)

Train size: 748, validation size: 188
Train size: 416, validation size: 104


# 2. Training

At the end of training we need:
1. a model compatible with scikit-learn api
2. (maybe???) an encoder

In [14]:
rfr_sj = train_rfr(	X_train_sj, y_train_sj)

y_pred_sj = predict_val(rfr_sj, X_val_sj, y_val_sj)

rfr_iq = train_rfr(X_train_iq, y_train_iq)

y_pred_iq = predict_val(rfr_iq, X_val_iq, y_val_iq)

26.762673931438712
7.917392467948717


# 3. Predict for test features and format for submition

For that we need:
1. [from precrocessing] - (list of) test DFs
2. [from training]      - (list of) models

In [15]:
get_submitable([test_sj, test_iq], [rfr_sj, rfr_iq], train_cols)

In [16]:
# target['date'] = pd.to_datetime(target['year'].astype('str') + '-' + target['weekofyear'].astype('str') + '-0', format='%Y-%W-%w')